# Run analysis on the data

In [37]:
%matplotlib qt

In [2]:
%run -i ~/Documents/tomology/tomography/utils.py

/home/xf28id2/conda_envs/khalifah_20210725/lib/python3.9/site-packages/pims/image_reader.py:26: RuntimeWarning: PIMS image_reader.py could not find scikit-image. Falling back to matplotlib's imread(), which uses floats instead of integers. This may break your scripts. 
(To ignore this warning, include the line "warnings.simplefilter("ignore", RuntimeWarning)" in your script.)
  warnings.warn(RuntimeWarning(ski_preferred))


In [3]:
import pyFAI
import pathlib

from ipywidgets import interact_manual

## Create the calculator

In [4]:
C = Calculator()

## Load the Run Data

In [11]:
from databroker import catalog

db = catalog["xpdd"]

In [12]:
UID = 'cb427382-2015-41b8-8d6d-aad41d84488d'
run = db[UID]
C.metadata = dict(run.metadata["start"])

In [13]:
data = run.primary.to_dask()
C.frames_arr = data["dexela_image"]
C.frames_arr

,Array,Chunk
Bytes,353.55 GiB,91.12 MiB
Shape,"(3973, 1, 3888, 3072)","(1, 1, 3888, 3072)"
Count,11919 Tasks,3973 Chunks
Type,float64,numpy.ndarray


In [16]:
C.metadata["shape"] = C.metadata["shape"][1:]
C.metadata["extents"] = C.metadata["extents"][1:]

In [17]:
# show the total intensity distribution
total_i = data["dexela_stats1_total"].values.reshape(C.metadata["shape"])
fig, ax = plt.subplots()
xr.DataArray(total_i).plot.imshow(ax=ax)
ax.invert_yaxis()
plt.show()

## Load the calibration data

In [18]:
PONI_FILE = "/nsls2/data/xpd/legacy/processed/from_mehmet/CeO2_0.25x0.25_beam.poni"
C.ai = pyFAI.load(PONI_FILE) if PONI_FILE else pyFAI.azimuthalIntegrator.AzimuthalIntegrator(dist=200, wavelength=0.186, detector="perkin")

## Choose the Max image

In [7]:
interact_manual(
    show_npy_array, 
    template="/nsls2/data/xpd/legacy/processed/xpdUser/tiff_base/CG-0032_cb427382-2015-41b8-8d6d-aad41d84488d/max/CG-0032_{}_max.npy",
    index=(1, 137 * 29)
)

interactive(children=(Text(value='/nsls2/data/xpd/legacy/processed/xpdUser/tiff_base/CG-0032_cb427382-2015-41b…

<function __main__.show_npy_array(template: str, index: int, **kwargs)>

In [22]:
INDEX = 3732
sample_name = C.metadata["sample_name"]
uid = C.metadata["uid"]
light_file = "/nsls2/data/xpd/legacy/processed/xpdUser/tiff_base/{}_{}/max/{}_{}_max.npy".format(sample_name, uid, sample_name, INDEX)
dark_file = "/nsls2/data/xpd/legacy/processed/xpdUser/tiff_base/{}_{}/min/{}_{}_min.npy".format(sample_name, uid, sample_name, INDEX)
C.light = np.load(light_file)
C.dark = np.load(dark_file)

## Subtract the dark from the light

In [23]:
C.light -= C.dark

## Locate the peaks and create windows

In [8]:
# tp.locate?

In [24]:
C.calc_peaks_from_light_frame(25, separation=10, threshold=30, topn=80)

In [27]:
C.calc_windows_from_peaks(num=80, width=50)
C.show_windows(size=10, vmax=400)
plt.show()

## Run throught the images to calculate the intensity in the windows

In [28]:
C.calc_intensity_in_windows()

In [29]:
C.reshape_intensity()
C.calc_coords()
C.assign_q_values()

In [32]:
import matplotlib as mpl

In [38]:
facet = C.show_intensity(col_wrap=20, sharey=False, norm=mpl.colors.TwoSlopeNorm(500))
axes = facet.axes.flatten()
for i in range(C.intensity.shape[0]):
    d = 2 * np.pi / C.windows["Q"]
    axes[i].set_title("d = {:.3f} nm".format(d.iloc[i]))
    axes[i].invert_yaxis()
    axes[i].set_aspect("equal")
plt.show()

## Save the result

In [39]:
BASE = "/nsls2/data/xpd/legacy/processed/xpdUser/tiff_base/"
FILENAME = pathlib.Path(BASE).joinpath("{}_{}.nc".format(C.metadata["sample_name"], C.metadata["uid"]))
C.to_dataset().to_netcdf(FILENAME)